# Interacting with Restricted Data in the Portal

<div>
<img alt="Screenshot of the Globus landing page" src="./img/access-restricted.png" style="width: 75%; height: 75%" />
</div>

Here we demonstrate how to create a page that allows a user to login to Globus and retrieve access tokens to then load restricted data they have access to into their browser. This example uses a single hardcoded CSV file, but it could be extended to be more flexible. Also, the login process could be move to the top level of your catalog so many pages could use the tokens.

The page we'll use, `chart-restricted.md` is a modified version of our previous JavaScript plotting example. The login capabilities are built on the [Globus JavaScript SDK](https://globus.github.io/globus-sdk-javascript/).

In [ ]:
import os
current_folder = %pwd
if not current_folder.endswith("cheapandfair-template"):
    %cd cheapandfair-template

In [ ]:
import toml
config = toml.load("config.toml")

In [ ]:
import os
github_username = os.environ["JUPYTERHUB_CLIENT_ID"].replace("jupyterhub-user-", "")

In [ ]:
github_username

## Registering an OAuth Application

To make this page work we'll need to register a client application with Globus Auth. This will allow the page to retrive tokens and user information so visitors to the page can interact with restricted data or call Globus services.

You can go directly to the [Globus Developers page under Settings](https://app.globus.org/settings/developers), or navigate to it by going to Settings on the left
<div>
<img alt="Screenshot of the Settings button in the Globus web app" src="./img/settings.png"/>
</div>
and then clicking on the "Developers" tab.
<div>
<img alt="Screenshot of the Settings page in the Globus web app" src="./img/devs.png"/>
</div>

On the Developers tab, click the option to register a thick client. In short, these are applications that run in user space, like apps on a smartphone. You may be prompted to consent again.
<div>
<img alt="Screenshot of the Developers tab in the Globus web app Settings" src="./img/thick-client.png" />
</div>

You will need to create a Globus Project, unless you have one for example applications. Give the project a name, provide a contact email address, and click continue.
<div>
<img alt="Screenshot of the Settings page in the Globus web app" src="./img/add-project.png"  style="width: 50%; height: 50%" />
</div>

### Get the Redirect URL

OAuth logins work by redirecting the user to a login page they trust, like their campus SSO or GitHub. After authenticating, Globus Auth will redirect the user's browser back to the application they are trying to login to. This means we need to provide Globus Auth one or more URLs that can be used for redirects. This prevents someone from redirecting a user to a malicious site after they've authenticated and potentially getting their tokens.

The next cell will print the URL of the page on your GitHub Pages site which is also the redirect location. 

In [ ]:
print(f'https://{github_username}.github.io/cheapandfair-template/chart-restricted.html')

### Register the App

On the App Registration page, give your App a name and enter the redirect URL from the previous cell. You don't need to change any other fields.

<div>
<img alt="Screenshot of the Globus Auth app registration page" src="./img/create-app.png"  style="width: 50%; height: 50%" />
</div>

### Get App UUID

After the App is registered, you'll see its details. Grab the Client UUID and put it in the cell below to update the `config.toml` file.

<div>
<img alt="Screenshot of a Globus Auth App details page" src="./img/app-details.png" style="width: 50%; height: 50%" />
</div>


In [ ]:
%%file -a config.toml
CLIENT='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

## Updating the Page

Now that we have the client application, we can update the Markdown for the page and then our GitHub Pages catalog site.

You can open the `chart-restricted.md` page in the Mardown editor [here](cheapandfair-template/chart-restricted.md) or in the file browser on the left.

### Linking to Your Group

There's a section of text near the top that allows a user to visit your Globus Group and request to join. Find the text near the top with a link to a Globus Group. 

<div>
<img alt="Screenshot of the script section of a Markdown file" src="./img/group-url.png"/>
</div>

Run the next cell to get the Group UUID and replace the `xxxx...` text in the URL with the UUID.

In [ ]:
print(config['GROUP'])

### Setting the Client ID, Data Location, Etc.

At the top of the `<script type="text/javascript">` section that makes up most of the file, there are four attributes you need to set:

- The UUID of your Collection so a token with the appropriate scope can be retrieved for the user.
- The OAuth client ID of your application.
- The redirect URL, which is the URL of the page.
- The URL of the restricted CSV file.

<div>
<img alt="Screenshot of the script section of a Markdown file" src="./img/update-md.png"/>
</div>

The cell below will print out all of the lines so you can copy and paste them into the Markdown file.

In [ ]:
config = toml.load("config.toml")

In [ ]:
print(f"const collection = \'{config['UUID']}\'")
print(f"const client_id = \'{config['CLIENT']}\'")
print(f"const redirect_url = \'https://{github_username}.github.io/cheapandfair-template/chart-restricted.html\'")
print(f"const csv_url = \'https://{config['DOMAIN']}{config['FOLDER']}cmb_spectra/cls.csv\'")

### Commit the Page and Push

In [ ]:
!git add chart-restricted.md

In [ ]:
!git commit -m "configure data access for restricted data"

In [ ]:
!git push

## Testing the Page

To test the page, evaluate the next cell and click on the link.

In [ ]:
print(f'https://{github_username}.github.io/cheapandfair-template/chart-restricted.html')

### Login

Click the login button.

<div>
<img alt="Screenshot of the Settings page in the Globus web app" src="./img/page-login.png"  style="width: 50%; height: 50%" />
</div>

### No Access

Users who don't have access can still login. They'll be prompted to request membership in your Globus Group. Since you own the Group and have access the data, you shouldn't see this.

<div>
<img alt="Screenshot of the Settings page in the Globus web app" src="./img/no-access.png"  style="width: 50%; height: 50%" />
</div>

### Success!

<div>
<img alt="Screenshot of the Settings page in the Globus web app" src="./img/restricted-plot.png"  style="width: 50%; height: 50%" />
</div>



